In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import concatenate, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = np.load('data/train.npy', allow_pickle = 'True')
test = np.load('data/test.npy', allow_pickle = 'True')

In [3]:
x = train[:,2:]
x = np.reshape(x, (-1, 28, 28, 1))

x_letter = train[:,1]
x_letter = np.reshape(x_letter, (-1, 1))
en = OneHotEncoder()
x_letter = en.fit_transform(x_letter).toarray()

y = train[:,0]
y = np.reshape(y, (-1, 1))
en = OneHotEncoder()
y = en.fit_transform(y).toarray()

print(x.shape)
print(x_letter.shape)
print(y.shape)

(2048, 28, 28, 1)
(2048, 26)
(2048, 10)


In [4]:
valid_size = 48
valid_x = x[-valid_size:]
x = x[:-48]

valid_x_letter = x_letter[-valid_size:]
x_letter = x_letter[:-48]

valid_y = y[-valid_size:]
y = y[:-48]

print(x.shape)
print(valid_x.shape)
print(x_letter.shape)
print(valid_x_letter.shape)
print(y.shape)
print(valid_y.shape)

(2000, 28, 28, 1)
(48, 28, 28, 1)
(2000, 26)
(48, 26)
(2000, 10)
(48, 10)


In [5]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     shear_range=10)

In [6]:
x_total = x.copy()
def augment(x):
    aug_list = []
    for i in range(x.shape[0]):
        num_aug = 0
        tmp = x[i]
        tmp = tmp.reshape((1,) + tmp.shape)
        for x_aug in image_generator.flow(tmp, batch_size = 1) :
            if num_aug >= 1:
                break
            aug_list.append(x_aug[0])
            num_aug += 1
    aug_list = np.array(aug_list)
    return aug_list

n = 2
for i in range(n):
    arr = augment(x)
    x_total = np.concatenate((x_total, arr), axis=0)
    if i > n:
        break

print(x_total.shape)

(6000, 28, 28, 1)


In [7]:
y_total = y.copy()
for i in range(n):
    arr = y.copy()
    y_total = np.concatenate((y_total, arr), axis=0)

print(y_total.shape)

(6000, 10)


In [8]:
x_letter_total = x_letter.copy()
for i in range(n):
    arr = x_letter.copy()
    x_letter_total = np.concatenate((x_letter_total, arr), axis=0)
    
print(x_letter_total.shape)

(6000, 26)


In [9]:
x_train, x_val, y_train, y_val = train_test_split(x_total, y_total, test_size=0.2, shuffle=True)#, stratify=y_total)
x_letter_train = x_letter_total[:x_train.shape[0],:]
x_letter_val = x_letter_total[x_train.shape[0]:,:]

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)
print(x_letter_train.shape)
print(x_letter_val.shape)

(4800, 28, 28, 1)
(1200, 28, 28, 1)
(4800, 10)
(1200, 10)
(4800, 26)
(1200, 26)


In [10]:
input1 = Input(shape=(28,28,1))
x1 = Conv2D(64, (3,3), activation='relu', padding='same')(input1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(128, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Flatten()(x1)

input2 = Input(shape=(26,))
merge = concatenate([x1, input2])

x2 = Dense(500, activation='relu')(merge)
x2 = Dropout(0.3)(x2)
x2 = Dense(100, activation='relu')(x2)
x2 = Dropout(0.3)(x2)
x2 = Dense(50, activation='relu')(x2)
outputs = Dense(10, activation='softmax')(x2)

model = Model(inputs = [input1, input2], outputs = outputs)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 28, 28, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 64)   0           dropout[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (

In [11]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')

#es = EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
cp = ModelCheckpoint('./models/{epoch:02d}-{val_acc:.4f}.h5', monitor='val_loss',
                     save_best_only=True, mode='min')

history = model.fit([x_train, x_letter_train], y_train,
                    validation_data=([x_val, x_letter_val], y_val), 
                    batch_size=64, epochs=100, verbose=1, callbacks = [cp])

Train on 4800 samples, validate on 1200 samples
Epoch 1/100
4800/4800 [==============================] - 5s 1ms/step - loss: 4.9813 - acc: 0.0983 - val_loss: 2.3009 - val_acc: 0.1042
Epoch 2/100
4800/4800 [==============================] - 2s 339us/step - loss: 2.3051 - acc: 0.1102 - val_loss: 2.2972 - val_acc: 0.1008
Epoch 3/100
4800/4800 [==============================] - 2s 340us/step - loss: 2.2909 - acc: 0.1185 - val_loss: 2.2914 - val_acc: 0.1175
Epoch 4/100
4800/4800 [==============================] - 2s 347us/step - loss: 2.2803 - acc: 0.1377 - val_loss: 2.2748 - val_acc: 0.1500
Epoch 5/100
4800/4800 [==============================] - 2s 342us/step - loss: 2.2569 - acc: 0.1508 - val_loss: 2.2516 - val_acc: 0.1767
Epoch 6/100
4800/4800 [==============================] - 2s 338us/step - loss: 2.2346 - acc: 0.1667 - val_loss: 2.1894 - val_acc: 0.2217
Epoch 7/100
4800/4800 [==============================] - 2s 331us/step - loss: 2.1705 - acc: 0.2015 - val_loss: 2.1365 - val_acc: 0.

4800/4800 [==============================] - 2s 344us/step - loss: 0.4567 - acc: 0.8458 - val_loss: 0.7537 - val_acc: 0.7675
Epoch 61/100
4800/4800 [==============================] - ETA: 0s - loss: 0.4337 - acc: 0.856 - 2s 342us/step - loss: 0.4326 - acc: 0.8567 - val_loss: 0.8615 - val_acc: 0.7142
Epoch 62/100
4800/4800 [==============================] - 2s 338us/step - loss: 0.4163 - acc: 0.8637 - val_loss: 0.7764 - val_acc: 0.7550
Epoch 63/100
4800/4800 [==============================] - 2s 345us/step - loss: 0.3891 - acc: 0.8754 - val_loss: 0.8170 - val_acc: 0.7225
Epoch 64/100
4800/4800 [==============================] - 2s 345us/step - loss: 0.3914 - acc: 0.8748 - val_loss: 0.7278 - val_acc: 0.7808
Epoch 65/100
4800/4800 [==============================] - 2s 338us/step - loss: 0.3983 - acc: 0.8708 - val_loss: 0.7112 - val_acc: 0.7825
Epoch 66/100
4800/4800 [==============================] - 2s 317us/step - loss: 0.3844 - acc: 0.8754 - val_loss: 0.7279 - val_acc: 0.7733
Epoch 67/

In [15]:
from tensorflow.keras.models import load_model
best_model = load_model('models/base_dropout.h5')

In [16]:
evaluate = best_model.evaluate([valid_x, valid_x_letter], valid_y)
evaluate

48/48 [==============================] - 0s 5ms/step


[0.854184110959371, 0.7083333333333334]

In [14]:
#submission = pd.read_csv('data/val.csv')
#submission['digit'] = np.argmax(best_model.predict([x_test, x_letter_test]), axis=1)
#submission.to_csv('data/val(CNN).csv', index=False)